# Back up an AllSpice Hub Repo

This example shows how to back up an AllSpice Hub repo as a zip file to a local directory. You can
then store the tar file to some other location, such as a cloud storage bucket.

This example only works with py-allspice v2.1.0 or later.

First, start by filling out the owner and name of the repo you want to back up here.

In [ ]:
# For example, if your repo is "test/test", then set these variables like this:
repo_owner = "test"
repo_name = "test"

In [ ]:
from allspice import AllSpice, Repository

# Put your AllSpice Hub token here.
allspice_client = AllSpice(token_text="<YOUR TOKEN HERE>")

# If you are self-hosting AllSpice Hub, you can input your url too:

# allspice_client = AllSpice(allspice_hub_url="<YOUR URL HERE>", token_text="<YOUR TOKEN HERE>")

In [ ]:
repo = Repository.request(allspice_client, repo_owner, repo_name)

with open(f"{repo_owner}_{repo_name}.zip", "wb") as f:
    data = repo.get_archive()
    f.write(data)

By default, this will fetch the `main` branch of your repo. If you want to change which branch or
commit you back up, you can use the `ref` argument to `get_archive`, which can take:

1. A branch name,
2. A commit hash,
3. A `Branch` or `Commit` object, which you can get from py-allspice.

In [ ]:
with open(f"{repo_owner}_{repo_name}.zip", "wb") as f:
    data = repo.get_archive(ref="my-branch")
    f.write(data)

In [ ]:
with open(f"{repo_owner}_{repo_name}.zip", "wb") as f:
    data = repo.get_archive(ref="my-commit-hash")
    f.write(data)

If you want to back up all current branches, you can fetch them from the repo and back them up one
by one:

In [ ]:
branches = repo.get_branches()
for branch in branches:
    with open(f"{repo_owner}_{repo_name}_{branch.name}.zip", "wb") as f:
        data = repo.get_archive(ref=branch)
        f.write(data)

And you can export as a tar.gz file instead of a zip file using the `archive_format` argument. This
can be mixed with the `ref` argument, if you want.

In [ ]:
with open(f"{repo_owner}_{repo_name}.tar.gz", "wb") as f:
    data = repo.get_archive(archive_format=Repository.ArchiveFormat.TAR)
    f.write(data)

Once you have the backed up file, you might want to upload it to a cloud storage bucket. Here's an
example of how to do that with e.g. S3:

In [ ]:
import boto3

s3 = boto3.client("s3")
s3.upload_file(
    f"{repo_owner}_{repo_name}.zip",
    "<YOUR BUCKET NAME HERE>",
    f"{repo_owner}_{repo_name}.zip",
)
